# Introduction

Previously in part 3, we tried multiple models and settled on the Support Vector Machine (SVM) as the best model to go forward with. In addition to performing the best of all models selected in part 3, SVM has many parameters to tune which lends itself well to exploration. In the previous part, our models were unable to beat the single feature baseline. I suspect that with more advanced hyperparameter tuning and with more shuffles of the data we will be able to notably improve model performance.

# Load the data
From part 3, we tried many variations of the training dataset. Here, I have settled on the min-max scaled dataset because it performed better in most scenarios from part 3. Moreover, SVM is sensitive to data scaling and using the scaled dataset did improve training time upon initial testing.

In [31]:
import pandas as pd
import numpy as np

In [32]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("./TFRRS-API/data/processed/raw.csv")
df.drop(["school", "q1", "q3", "grade"], axis=1, inplace=True)

le = LabelEncoder()
df["event_num"] = le.fit_transform(df["event"])

X_scaled = pd.read_csv("./TFRRS-API/data/processed/scaled.csv")
y_all_am = df["all_american"]

# Train Test split
Recall at our data consists of 18 events which were split 13/5 to form the train test data. Because we are using the random library and fixing the seed we obtain the same train/test split as used in part 3.

In [33]:
import random

random.seed(0)

test_numbers = random.choices(range(18), k=5)

test_indices = df[df["event_num"].isin(test_numbers)].index
X_test = X_scaled.iloc[test_indices]
y_test = y_all_am.iloc[test_indices]

train_indices = df[~df["event_num"].isin(test_numbers)].index
X_train = X_scaled.iloc[train_indices]
y_train = y_all_am.iloc[train_indices]

# Custom metric function

Recall that this model is aiming to solve a ranking problem while treating all event groups equally. Using an arbitrary example, it is possible for the 10th best 1500m runner to get a higher predicted probability than the 1st place Javelin thrower. This potential group imbalance between leads to sklearn's built-in metrics not being the most effective. Thus, I have implemented my custom metric function from part 3 as a scoring function for cross validation.

This function will take predictions and, based on the training data index, map the values to our original data. From there, places gets the eight highest predicted athletes within each event group and counts what percentage of them were actually all-americans.

In [36]:
from sklearn.metrics import make_scorer

# Function from part 3
def percent_all_american_score(y_true, y_pred, indices):  # =df.index):
    truth = df.iloc[indices][["name", "event", "place", "all_american"]].copy()
    truth["prob_all_am"] = y_pred
    top_eight = (
        truth.sort_values(["event", "prob_all_am"], ascending=False)
        .groupby(["event"])
        .head(8)
    )
    return np.mean(top_eight["all_american"])


# Make it compatible with sklearn and allow arbitrary X datasets
def test_score_func(train_x, y_pred):
    return percent_all_american_score(None, y_pred, indices=train_x.index)


my_scorer = make_scorer(test_score_func, needs_proba=True)

# Initial Hyperparameter Selection
When evaluating SVM as our model, we must consider the options of kernels to select. As we saw in part 3, the sigmoid kernel performed much notably worse than other options so I am leaving it out. Due to the performance considerations of our cross validation involving many splits, I will be doing an initial grid search between linear, polynomial, and rbf to determine the best kernel to optimize further. Moreover, I will use a simple range of C and the two non-float gamma arguments, for the appropriate kernels, to test the kernels under some variation.

With the hyperparameter strategy decided, recall that the dataset has 18 groups which are split into 13/5 for train/test. I will test the hyperparameters by further making a 10/3 split for train/validation and averaging model performance over the possible permutation of the train/validation groups. We need to average over many splits of data because we only have one year's worth of data so the model can be sensitive to which events it was trained on.

In [28]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, LeavePGroupsOut
import joblib

In [29]:
C_vals = [10**x for x in range(-2, 2)]
gamma_vals = ["auto", "scale"]

params = [
    # Linear
    {"kernel": ["linear"], "C": C_vals},
    # Poly
    {
        "kernel": ["poly"],
        "degree": range(2, 5),
        "C": C_vals,
        "coef0": [0, 0.5, 1],
        "gamma": gamma_vals,
    },
    # Rbf
    {"kernel": ["rbf"], "C": C_vals, "gamma": gamma_vals},
]

In [30]:
# Using support vector classifier as our model
svm = SVC(probability=True, class_weight="balanced", random_state=0)

# This creates the 10/3 split
num_groups = 3
splitter = LeavePGroupsOut(n_groups=num_groups)

# Create the grid search object
clf = GridSearchCV(
    estimator=svm,
    param_grid=params,
    scoring=my_scorer,
    n_jobs=4,
    cv=splitter,
    verbose=10,
)

In [31]:
# Fit the model and save results
clf.fit(X_train, y_train, groups=df.iloc[X_train.index]["event_num"])
joblib.dump(clf, f"./models/svm_cross_val.pkl", compress=True)

Fitting 286 folds for each of 84 candidates, totalling 24024 fits


['./models/svm_cross_val.pkl']

In [32]:
results = {
    item: val for item, val in clf.cv_results_.items() if not item.startswith("split")
}

# Turn the results into a dataframe for visualization
# Show the 10 best models found during grid search
cv_results = pd.DataFrame(results)
cv_results.sort_values(["mean_test_score", "std_test_score"], ascending=[False, True])[
    [
        "mean_test_score",
        "std_test_score",
        "param_kernel",
        "param_C",
        "param_degree",
        "param_coef0",
        "param_gamma",
    ]
].head(10)

,mean_test_score,std_test_score,param_kernel,param_C,param_degree,param_coef0,param_gamma
41,0.591200,0.117019,poly,1,2,0,scale
58,0.587558,0.102893,poly,10,2,0,auto
46,0.586393,0.122187,poly,1,2,0.5,auto
47,0.585810,0.114082,poly,1,2,0.5,scale
52,0.585664,0.119465,poly,1,2,1,auto
53,0.580128,0.110523,poly,1,2,1,scale
48,0.579254,0.108778,poly,1,3,0.5,auto
2,0.578817,0.100312,linear,1,NaN,NaN,NaN
54,0.576632,0.109372,poly,1,3,1,auto
37,0.574009,0.118459,poly,0.1,3,1,scale


# Initial Analysis
In line with what we determined in part 3, the polynomial kernel is clearly the best for this model. Moreover, degree 2 is clearly the best performing degree with it taking six of the top ten spots. This is encouraging news because while higher degree models are more complex thus can fit the data better they can also lead to overfitting. One thing to note that the model performance here is actually worse than what was found in part 3. However, this is not concerning because as mentioned previously the small dataset makes the model dependent on the random shuffle; during training I observed that the model did score over 0.7 on individual validation sets however it's the average performance on unknown data we care about.

# Further tuning strategy
Now that we know which kernel is best for the data, as well as knowing what degree to use, we will optimize the model further by trying more granular values of C and gamma. Given that the best and second-best model performances were with C=1 and C=10 respectively I will include more values of C in that range. Regarding gamma, I noticed that large values of gamma significantly slow down training when we have so many splits thus they are not feasible for cross validation on my machine. Luckily, if you look ahead you will notice that the best performance comes with smaller values of gamma regardless.

In [33]:
optimized = {
    "C": [0.1, 0.5, 1, 3, 5, 10],
    "coef0": [0, 0.5, 1],
    "gamma": [10**x for x in range(-5, 1)] + ["auto", "scale"],
}

# Fixed this to second degree polynomial SVM based on first tuning
svm2 = SVC(
    kernel="poly", degree=2, probability=True, class_weight="balanced", random_state=0
)

# Create the grid search object
clf2 = GridSearchCV(
    estimator=svm2,
    param_grid=optimized,
    scoring=my_scorer,
    n_jobs=4,
    cv=splitter,
    verbose=10,
)

In [34]:
# Train the model and save results
clf2.fit(X_train, y_train, groups=df.iloc[X_train.index]["event_num"])
joblib.dump(clf2, f"./models/final_svm_cross_val.pkl", compress=True)

Fitting 286 folds for each of 144 candidates, totalling 41184 fits


['./models/final_svm_cross_val.pkl']

In [35]:
# From the grid search results pull out data excluding the individual splits
optimized_results = {
    item: val for item, val in clf2.cv_results_.items() if not item.startswith("split")
}

# Turn the results into a dataframe for visualization
# Show the 10 best models found during grid search
df_opt = pd.DataFrame(optimized_results)
df_opt.sort_values(["mean_test_score", "std_test_score"], ascending=[False, True])[
    ["mean_test_score", "std_test_score", "param_C", "param_coef0", "param_gamma"]
].head(10)

,mean_test_score,std_test_score,param_C,param_coef0,param_gamma
55,0.591200,0.117019,1,0,scale
78,0.590618,0.116276,3,0,auto
139,0.589744,0.121284,10,1,0.01
100,0.589452,0.105566,5,0,0.1
76,0.588578,0.112945,3,0,0.1
79,0.588432,0.105072,3,0,scale
126,0.587558,0.102893,10,0,auto
102,0.586976,0.111739,5,0,auto
44,0.586976,0.122716,0.5,1,0.1
60,0.586830,0.118005,1,0.5,0.1


In [36]:
print(clf2.best_estimator_._gamma)

0.1262240553486386


# Final Analysis
Interestingly, the further hyperparameter tuning did not produce a model better than what we already found previously. That being said, we can still make observations incase we want to impose more regularization in hopes of giving a more generalizable model. The first item that stands out to me is how clearly values around 0.1 for gamma perform best with the model. Thus, we don't want to make large changes to gamma otherwise model performance will likely suffer. Regarding C, while my intuition of adding values between 1 and 10 paid off based on nine out of ten best models being in that range, fortunately the best model has C=1 so we don't have decrease regularization for a better fit.

Overall, we clearly found some pattern in the data based on our range of hyperparameters with the top 10 models all being within 1% accuracy of each other.

# Testing
Now, we will use our best model (which is the same from both grid search objects) to predict the test data.

In [37]:
preds = clf2.predict_proba(X_test)[:, 1]
print(percent_all_american_score(None, preds, indices=X_test.index))

0.675


Shockingly, we get a higher test accuracy than validation accuracy. I would attribute this to having so aforementioend relatively few data samples. For further analysis, let's dig into the errors and examine who the model missed as placing all-american

In [40]:
import joblib
clf = joblib.load("models/final_svm_cross_val.pkl")
preds = clf.predict_proba(X_test)[:, 1]

model_test_df = df.iloc[X_test.index].copy()
model_test_df["prob_all_am"] = preds

# Get the top eight in each event
model_top_eight = (
    model_test_df.sort_values(["event", "prob_all_am"], ascending=False)
    .groupby(["event"])
    .head(8)
)
model_top_eight[["name", "event", "place", "all_american", "prob_all_am"]]

,name,event,place,all_american,prob_all_am
263,CLAYTON FRITSCH,PV,2,1,0.666345
270,ZACH BRADFORD,PV,9,0,0.516245
267,ZACH MCWHORTER,PV,6,1,0.468099
262,SONDRE GUTTORMSEN,PV,1,1,0.445348
266,BRANSON ELLIS,PV,5,1,0.411566
272,CALEB WITSKEN,PV,11,0,0.364169
274,EERIK HAAMER,PV,13,0,0.350894
271,TREVOR STEPHENSON,PV,10,0,0.336603
407,ETHAN DABBS,JT,2,1,0.289275
406,MARC MINICHELLO,JT,1,1,0.235698


In [38]:
# append predictions to a copy of the test data
model_test_df = df.iloc[X_test.index].copy()
model_test_df["prob_all_am"] = preds

# Get the top eight in each event
model_top_eight = (
    model_test_df.sort_values(["event", "prob_all_am"], ascending=False)
    .groupby(["event"])
    .head(8)
)

# Find all americans which were not in our predicted top eight
missed_all_americans = model_test_df[
    (~model_test_df["name"].isin(model_top_eight["name"]))
    & (model_test_df["all_american"] == 1)
][["name", "event", "place", "prob_all_am"]].sort_values("place")
missed_all_americans

,name,event,place,prob_all_am
72,MOAD ZAHAFI,800,1,0.306351
26,UDODI ONWUZURIKE,200,3,0.401963
264,KEATON DANIEL,PV,3,0.247668
195,NATHANIEL EZEKIEL,400H,4,0.378536
265,SIMEN GUTTORMSEN,PV,4,0.321034
28,TARSIS OROGOT,200,5,0.417426
76,SEAN DOLAN,800,5,0.398337
77,DAYTON CARLSON,800,6,0.395270
30,ROBERT GREGORY,200,7,0.476354
268,CLAYTON SIMMS,PV,7,0.237704


The missed predictions are quite uniform in terms of events and placement positions missed. There is no clear pattern in the false negative samples.


The test data performance is better than expected and has gotten closer to the training baseline accuracy of nearly 73% using a single feature. Now, if we use that same baseline on the test data we obtain

In [39]:
column_test_df = df.iloc[X_test.index].copy()
column_top_eight = (
    model_test_df.sort_values(["event", "mean_three_best"], ascending=False)
    .groupby(["event"])
    .head(8)
)
np.mean(column_top_eight["all_american"])

0.825

which follows the trends of doing better on the test data and being better than our model.

# Conclusion

The conclusion to this project is that machine learning was not well suited to solve this problem. The models I trained did outperform random guessing but they were heavily carried by our singular feature baseline. While I am glad that my subject area expertise of track and field allowed me to create such a useful singular feature, I am also surprised that adding information quickly worsens model performance. I suspect that with more years of data, the dataset will be more resilient while also ena new features based on historical championship performance. Regardless, this was a good exercise in machine learning as I was able to explore multiple models and effectively tune hyperparameters to improve the performance of the ill-fated model.

### Checking assumption of group independence
While the following does not affect my conclusions on the model, I wanted to do further analysis for my own interest

In part 1 of the project, I set forth an assumption that we can model all events equally. Because the smaller event groups only have 54 samples, creating an ensemble model based on event groups would have uncomfortably high variance. However, I want to use the currently trained model to check the assumption that event groups can be modeled equally.

In [40]:
# Define the event groups and invert the selection for mapping
event_groups = {
    "sprints": ["100", "110H", "200", "400"],
    "mid distance": ["400H", "800", "1500"],
    "distance": ["3000S", "5000", "10000"],
    "jumps": ["HJ", "PV", "LJ", "TJ"],
    "throws": ["SP", "DT", "HT", "JT"],
}
inverse_events = {v: k for k, l in event_groups.items() for v in l}

In [19]:
# Predict the entire dataset
temp = df[["place", "all_american", "event", "name"]].copy()
all_preds = clf2.predict_proba(X_scaled)[:, 1]

# Map the event to its group
temp["prob_all_am"] = all_preds
temp["event_group"] = df["event"].apply(lambda x: inverse_events[x])

# Note where the model missed all americans
top_eight = (
    temp.sort_values(["event", "prob_all_am"], ascending=False)
    .groupby(["event"])
    .head(8)
)
missed = temp[(~temp["name"].isin(top_eight["name"])) & (temp["all_american"] == 1)][
    ["name", "event", "place", "prob_all_am", "event_group"]
]

# Average missed by number of events in event group
average_misses = pd.DataFrame(
    missed.groupby("event_group")["event"]
    .count()
    .sort_values()
    .reset_index()
    .rename({"event": "misses"}, axis=1)
)
average_misses["misses_per_event"] = average_misses.apply(
    lambda x: round(x["misses"] / len(event_groups[x["event_group"]]), 2), axis=1
)
average_misses[["event_group", "misses_per_event"]].sort_values("misses_per_event")

,event_group,misses_per_event
1,throws,2.00
0,distance,2.67
2,mid distance,3.00
3,jumps,3.00
4,sprints,3.00


While these results are not dramatically different I do think they hint at the assumption of model independence breaking down. Throws are the most distinct track and field event and coincidentally had the fewest errors. Given that jumps are also distinct from the running events, why doesn't the model perform better on jumps as well? Regardless more data would be needed as this year could have just had clear favorites in throws while having tightly contested running events.